# Example: Multiple Tables
In this example, I show how doctable can be used with multiple tables. To accomplish this, I first create a new DocTable object named SubTable that stores a single value `n` in addition to an index column, column recording when item was inserted, and an index to ensure speed and uniqueness of the number `n`. Next, we create a MainTable object which includes a foreign key constraint, and overloads the `DocTable.insert()` method to store data into the SubTable transparently.

In [1]:
import sys
sys.path.append('..')
import doctable

First, we create a simple class SubTable. Nothing special here.

In [2]:
class SubTable(doctable.DocTable):
    tabname = 'subtable'
    def __init__(self, fname):
        super().__init__(schema = (
            ('idcol', 'id'),
            ('integer', 'n'),
            ('date_updated', 'updated'),
            ('index', 'ind_n', ['n'], dict(unique=True)),
        ), fname=fname, tabname=self.tabname)

Next, we create a class MainTable with a foreign key constraint. The coreign key points from it's own column subkey to the SubTab `id` column. Additionally, we overload the MainTable insert method to store the provided `n` value into the other table and instead insert a reference to the SubTable that stores the actual integer.

In [3]:
class MainTable(doctable.DocTable):
    tabname = 'maintable'
    def __init__(self, fname):
        self.subtab = SubTable(fname)
        #self.subtab.commit()
        super().__init__(schema = (
            ('idcol', 'id'),
            ('integer', 'subkey'),
            ('string', 'name'),
            ('foreignkey_constraint', [['subkey'], [self.subtab['id']]], {}, dict(onupdate="CASCADE", ondelete="CASCADE")),
            ('date_updated', 'updated'),
        ), fname=fname, tabname=self.tabname)
    
    def insert(self, rowdat, **kwargs):
        # add just in case it wasn't there
        if 'n' in rowdat:
            self.subtab.insert({'n':rowdat['n']}, ifnotunique='ignore')
        
            # get id of the right column
            idx = self.subtab.select_first('id', where=self.subtab['n']==rowdat['n'])
            rowdat['subkey'] = idx
            del rowdat['n']
        super().insert(rowdat, **kwargs)

Now we test our code by creating a new MainTable object (which also instantiates a SubTable as a member variable), and then inserts two rows with different names and the same number. The result is that two rows have been inserted in MainTable but they contain references to the saim subtable rows.

In [5]:
db = MainTable('exdb/multitable0.db')
print(db)
for name in ('devin cornell', 'pierre bourdieu'):
    db.insert({'name':name, 'n':10})
print(db)
print(db.select_df())
print(db.subtab.select_df())

<DocTable::maintable ct: 0>
<DocTable::maintable ct: 2>
   id  subkey             name                    updated
0   1       1    devin cornell 2020-01-31 19:13:15.624948
1   2       1  pierre bourdieu 2020-01-31 19:13:15.628480
   id   n                    updated
0   1  10 2020-01-31 19:13:15.552712


Note how the output shows two rows created in the MainTable but only one in the SubTable. According to computational constraints one might choose not to perform the checks for table existance in the insert method, but in this case it is useful.

## Automatically Join Using Select
The great part about using sqlalchemy core is that you can manipulate multiple tables by accessing their doctable objects. In this case, we perform a join of name to their associated n (stored in the subtable) using the column subscript for the db.subtab member (of type SubTable). By default this does an outer join! Not sure what happens in a many-to-many table.

In [9]:
db.select(['name', db.subtab['n']])

[('devin cornell', 10), ('pierre bourdieu', 10)]